In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv("data/item_properties_part1.csv")
df2 = pd.read_csv("data/item_properties_part2.csv")
df = pd.concat([df1, df2])
df.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [3]:
replay_buffer = pd.read_pickle('data/replay_buffer.df')

In [4]:

s = set()
for i in replay_buffer["state"]:
    for j in i:
        s.add(j)
        
len(s)

68309

In [5]:
all_item = set(pd.unique(replay_buffer["action"]))

In [6]:
(replay_buffer["action"] == 1).sum()

11

In [7]:
df_filtered = df.loc[df["itemid"].isin(all_item)]

In [8]:
df_filtered

,timestamp,itemid,property,value
3,1431226800000,59481,790,n15360.000
18,1437274800000,49337,0,n36.000 1186610 119932 717520 903287 98606 632...
20,1439694000000,48696,566,n480.000 639502 189174
26,1435460400000,16615,888,150169 176547 824301 24474 293011 1240134
44,1438484400000,38351,544,1055803 1164666
...,...,...,...,...
9275876,1439694000000,4849,888,297765 406279 907471 88645 279913 1223352 1318567
9275883,1435460400000,10107,505,769062
9275887,1433646000000,9360,987,166332 n2520.000 377738
9275889,1434250800000,12864,888,249879 724834 1172269


In [9]:
# unstatic = set()
# count = 0
# for item in s:
#     cur_item = df_filtered.loc[df_filtered["itemid"] == item]
#     cur_property = cur_item["property"].value_counts()
#     for prop in cur_property.index:
#         if cur_property[prop] > 1:
#             unstatic.add(prop)
#     count += 1
#     if count % 1000 == 0:
#         print(count)
# unstatic
with open("unstatic_feature.txt", "r") as f:
    s = f.read()

s = s[1:-1].split(", ")
s1 = []
for c in s:
    s1.append(c[1:-1])
    # if c[0].isdigit():
    #     s1.append(int(c))
    # else:
    #     s1.append(c)
unstatic = set(s1)
    

In [10]:
df_filtered = df_filtered.loc[~df_filtered["property"].isin(unstatic)]
df_filtered

,timestamp,itemid,property,value
209,1433041200000,35535,671,769062
218,1435460400000,10394,631,n360.000 988555
231,1431226800000,15401,159,519769
234,1433646000000,63632,264,150169 153715 484436
327,1433646000000,62921,830,373898
...,...,...,...,...
9275746,1433041200000,48460,112,679677
9275800,1431226800000,15994,159,519769
9275818,1436670000000,44621,259,185669 69507
9275839,1440298800000,52102,591,1116693


In [11]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
categorical_features = ['property', 'value']
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoded = one_hot_encoder.fit_transform(df_filtered[categorical_features])
one_hot_encoded.shape

/Users/yiyang/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(547537, 20817)

In [15]:
max(df_filtered["itemid"])

70851

In [ ]:
item_array = np.array(list(all_item))
item_array
idx = np.arange(70852)
features = np.zeros((70852, one_hot_encoded.shape[1]))


In [ ]:

for i in range(100000):
    if i % 10000 == 0:
        print(i)
    row = df_filtered.iloc[i,:]
    itemid = row["itemid"]
    features[itemid] += one_hot_encoded[i]
    

In [23]:

with open('item_feature_300000.npy', 'wb') as f:
    np.save(f, features)


In [ ]:
with open('item_feature_300000.npy', 'rb') as f:
    features = np.load(f)

In [22]:

# df_filtered.shape[0]
for i in range(500000,df_filtered.shape[0]):
    if i % 10000 == 0:
        print(i)
    row = df_filtered.iloc[i,:]
    itemid = row["itemid"]
    features[itemid] += one_hot_encoded[i]
    

500000
510000
520000
530000
540000


In [24]:
features.sum()

1095074.0

In [27]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)

# Fit PCA on your data
pca.fit(features)

# Transform the data according to the PCA
X_reduced = pca.transform(X)

: 